# Sentiment Analysis with Deep Learning using BERT

## Introduction

### What is BERT

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.

For more information, the original paper can be found [here](https://arxiv.org/abs/1810.04805). 

[HuggingFace documentation](https://huggingface.co/transformers/model_doc/bert.html)

[Bert documentation](https://characters.fandom.com/wiki/Bert_(Sesame_Street) ;)

## Exploratory Data Analysis and Preprocessing

We will use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [130]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [131]:
df = pd.read_csv('/content/smileannotationsfinal.csv',
                 names=['id', 'text','category'])
df.set_index('id', inplace=True)
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [132]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [133]:
df = df[~df.category.str.contains('\|')]
df = df[df.category != 'nocode']

In [134]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [135]:
possible_labels = df.category.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [136]:
label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [137]:
df['label'] = df.category.replace(label_dict)
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


## Training/Validation Split

In [138]:
from sklearn.model_selection import train_test_split

In [139]:
X_train, X_val, y_train, y_val = train_test_split(
                                df.index.values,
                                df.label.values,
                                test_size = 0.15,
                                random_state=17,
                                stratify = df.label.values)

In [140]:
df['data_type'] = ['not_set']*df.shape[0]

In [141]:
df.head()

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not_set


In [142]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

## Loading Tokenizer and Encoding our Data

In [ ]:
pip install transformers

In [144]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [145]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                         do_lower_case=True)

In [146]:
encode_data_train = tokenizer.batch_encode_plus(
                        df[df.data_type == 'train'].text.values,
                        add_special_tokens=True,
                        return_attention_mask=True,
                        pad_to_max_length=256,
                        return_tensors='pt')


encode_data_val = tokenizer.batch_encode_plus(
                        df[df.data_type == 'val'].text.values,
                        add_special_tokens=True,
                        return_attention_mask=True,
                        pad_to_max_length=256,
                        return_tensors='pt')

input_ids_train = encode_data_train['input_ids']
attention_masks_train= encode_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)



input_ids_val = encode_data_val['input_ids']
attention_masks_val= encode_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [147]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [148]:
len(dataset_train)

1258

## Setting up BERT Pretrained Model

In [149]:
from transformers import BertForSequenceClassification

In [150]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', 
                                        num_labels=len(label_dict),
                                     output_attentions=False,
                                    output_hidden_states=False
                                     )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Creating Data Loaders

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [151]:

from torch.utils.data import DataLoader, RandomSampler
batch_size=4
dataloader_train = DataLoader(
                    dataset_train,
                    sampler=RandomSampler(dataset_train),
                    batch_size=batch_size)

dataloader_val = DataLoader(
                    dataset_val,
                    sampler=RandomSampler(dataset_val),
                    batch_size=32)


## Setting Up Optimizer and Scheduler

In [152]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [153]:
optimizer = AdamW(
                model.parameters(),
                lr=1e-5,
                eps=1e-8)

In [154]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=0,
                num_training_steps= len(dataloader_train)*epochs)

## Defining our Performance Metrics

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [155]:
import numpy as np

In [156]:
from sklearn.metrics import f1_score

In [157]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat,preds_flat,average='weighted')

In [158]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v:k for k,v in label_dict.items()}
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true= labels_flat[labels_flat == label]
        print(f'Class : {label_dict_inverse[label]}')
        print(f"Accuracy : {len(y_preds[y_preds==label])}/{len(y_true)}\n")

## Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [159]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [160]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)
print(device)

cuda


In [161]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [162]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch),
                                        leave=False,
                                        disable=False)
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids':batch[0],
            'attention_mask':batch[1],
            'labels':batch[2]
        }
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(),1.0)
        optimizer.step()
        scheduler.step()
        progress_bar.set_postfix({"training_loss": '{:.3f}'.format(loss.item()/len(batch))})
        
    torch.save(model.state_dict(), f'/content/Models/BERT_ft_epoch{epoch}.model')
    tqdm.write(f'\nEpoch {epoch}')
    loss_train_avg  = loss_train_total/len(dataloader_train)
    
    tqdm.write(f'Training_loss : {loss_train_avg}')
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation_loss : {val_loss}')
    tqdm.write(f'F1 score : {val_f1}')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.



Epoch 1
Training_loss : 0.7213589424710898



Validation_loss : 0.68238194499697
F1 score : 0.7827842801387063



Epoch 2
Training_loss : 0.42582314984045094



Validation_loss : 0.6829125881195068
F1 score : 0.8333855820736844


...


Validation_loss : 0.7038699663576803
F1 score : 0.8753037180926665



Epoch 9
Training_loss : 0.031209372952561973



Validation_loss : 0.7089231769953456
F1 score : 0.8753037180926665



Epoch 10
Training_loss : 0.02352711165504944



Validation_loss : 0.7161471971443721
F1 score : 0.8666034264688974



## Loading and Evaluating our Model

Load the model with best F1 Score and lets predict 

In [163]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [164]:
model.to(device)
pass

In [171]:
model.load_state_dict(
    torch.load("/content/Models/BERT_ft_epoch8.model",
              map_location=torch.device('cpu')))

<All keys matched successfully>

In [172]:
_,predictions, true_vals = evaluate(dataloader_val)

In [173]:
accuracy_per_class(predictions, true_vals)

Class : happy
Accuracy : 165/171

Class : not-relevant
Accuracy : 23/32

Class : angry
Accuracy : 6/9

Class : disgust
Accuracy : 0/1

Class : sad
Accuracy : 1/5

Class : surprise
Accuracy : 2/5

